## Code to calculate the centroids of the (trimmed) terminus picks

#### Jukes Liu

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.image as mpimg
import datetime
import types
import os
import cv2
import scipy.stats

If output images have not yet been converted to png format from pgm, do it using cell magic:

In [30]:
%%bash

cd /media/jukes/jukes1/LS8aws/Box033/rotated/resized/
mogrify -format png *.pgm

### 1) Set BoxIDs to do this for and define the centroid function and read in datetime df

In [31]:
BoxIDs = ['001', '002', '004', '033', '120', '174', '235', '259', '277', '531']

#Define the centroid function
def centroid(x, y):
    length = len(x)
    return sum(x) / length, sum(y) / length

In [38]:
#Read in datetags csv as datetime_df
datetime_df = pd.read_csv('/home/jukes/Documents/Sample_glaciers/datetags.csv', sep=',', dtype=str, header=0, names=['Img_Date', 'datetimes'])
datetime_df.head()

,Img_Date,datetimes
0,LC80360042017077LGN00,2017-03-18
1,LC80360042015248LGN00,2015-09-05
2,LC80360042015184LGN00,2015-07-03
3,LC80360042016107LGN00,2016-04-16
4,LC80360042015232LGN00,2015-08-20


### 2) Calculate centroids for terminus picked using a metric (mass or size)

Grabs the terminus pick line coordinates from the .dat files generated from the 2D WTMM in Xsmurf and calculates their centroid using the centroid function. This calculates the centroids for the original dat file (term_dat) AND the trimmed terminus pick (term_trim_dat). Currently only outputs the trimmed termini centroids.

In [39]:
centroid_xs = []
centroid_ys = []
BOIs_final = []
basepath = '/media/jukes/jukes1/LS8aws/'
massorsize = "mass"
metric = "terminus_highest"+massorsize+"/" 

for BOI in BoxIDs:
    print("Box"+BOI)
    imagepath = basepath+"Box"+BOI+"/rotated/resized/"

#     #make results directory in BoxID folder if it doesn't already exist
#     if os.path.exists(basepath+"Box"+BOI+"/Results/"):
#         print("RESULTS FOLDER EXISTS ALREADY. SKIP.")
#     #OTHERWISE, create the folder and download into it
#     else:
#         os.mkdir(basepath+"Box"+BOI+"/Results/")
#         print("Results  folder made")

    #make lists to store image data and grab image files
    imgfiles = os.listdir(imagepath)
    image_arrays = []
    dats = []
    trimdats = []
    imgnames = []
    avgpix_values = []
    skews = []
    BOIs =[]

    for imgfile in imgfiles:
        #grab image files and append to images list
        if imgfile.endswith(BOI+".png"):
    #         print(imgfile)
            image = mpimg.imread(imagepath+imgfile)
            imgname = imgfile[0:-4]
            scenename = imgname[7:-16]
            pathtodat = imagepath+imgname+".pgm_max_gaussian/"+metric
            datfiles = os.listdir(pathtodat)
            
        
            #NOT FILTERING FOR CLOUDS AGAIN CURRENTLY:
            #If pixel values are skewed toward 1, it's prob cloudy
            pixelvals = image.reshape(image.shape[0]*image.shape[1])
    #         print(pixelvals.shape)
            skew = scipy.stats.skew(pixelvals, bias=False)

            avgpix_val = np.average(pixelvals)
            avgpix_thresh = 0.50  

            #if there are 2 datfiles and not cloudy, grab the trimmed and non-trimmed files
            if len(datfiles) == 2: #and avgpix_val < avgpix_thresh and skew > -0.80:
                #append the image array and the image name to the list
                image_arrays.append(image)
                imgnames.append(scenename)
                skews.append(skew)
                avgpix_values.append(avgpix_val)
                BOIs.append(BOI)

                #find the trimmed dat file and the original
                for dat in datfiles:
                    if "trim" in dat:
                        datfile_trim = dat
                        trimdats.append(datfile_trim)
                    else:
                        datfile = dat
                        dats.append(datfile)

    #         print(image, datfile_trim, datfile)
#             else:
#                 print("NO DAT FILES CREATED FOR TERMINUS PICK")

    print(len(image_arrays), len(dats), len(trimdats), len(imgnames), len(avgpix_values))
    images_df = pd.DataFrame(list(zip(imgnames, BOIs, image_arrays, dats, trimdats, avgpix_values, skews)),
                  columns=['Scene', 'BoxID','Image array', 'Dat file name', "Trimmed dat file name", 'Avg pix val', 'Skew'])
    
    #JOIN DATAFRAMES
    images_df.sort_values(by='Scene')
    # images_df
    datetime_df = datetime_df.sort_values(by='Img_Date')
#     print(datetime_df.head())
    
    new_df = images_df.set_index('Scene').join(datetime_df.set_index('Img_Date'))
    dated_images_df = new_df.sort_values(by='datetimes')
#     print(dated_images_df.head())

    #CALCULATE ALL CENTROIDS
    for index, row in dated_images_df.iterrows():
        imagename = index
        dat = row['Trimmed dat file name']
        trimdat = row['Dat file name']
        BoxID = row['BoxID']
        
        datpath = basepath+"Box"+BoxID+"/rotated/resized/crop_R_"+imagename+"_B8_PS_Buffer"+BoxID+".pgm_max_gaussian/"+metric

        #Read in dat file as np array and grab x and y values
        #TRIMMED:
        term_trim_dat = np.loadtxt(datpath+trimdat)

        #ORIGINAL:
        term_dat = np.loadtxt(datpath+dat)

        #ORIGINAL
        term_xs = []
        term_ys = []

        #grab x and y values for the terminus line
        for j in term_dat:
            x, y = (j[0], j[1])
            term_xs.append(x)
            term_ys.append(y)

        #TRIMMED
        term_trim_xs = []
        term_trim_ys = []

        #grab x and y values for the terminus line
        for j in term_trim_dat:
    #         print(j)
            x, y = (j[0], j[1])
            term_trim_xs.append(x)
            term_trim_ys.append(y)

        #CALCULATE CENTROIDS AND APPEND TO LISTS
        center_x, center_y = centroid(term_xs, term_ys)
        trim_center_x, trim_center_y = centroid(term_trim_xs, term_trim_ys)
        centroid_xs.append(trim_center_x)
        centroid_ys.append(trim_center_y)
        BOIs_final.append(BoxID)

Box001
164 164 164 164 164
Box002
29 29 29 29 29
Box004
20 20 20 20 20
Box033
112 112 112 112 112
Box120
103 103 103 103 103
Box174
60 60 60 60 60
Box235
124 124 124 124 124
Box259
115 115 115 115 115
Box277
62 62 62 62 62
Box531
237 237 237 237 237


### 3) Store the centroids in a DataFrame and export to a csv file

Exports the trimmed terminus centroids to a csv file called __trim_centroids.csv__.

In [40]:
centroids_df = pd.DataFrame(list(zip(imgnames, BOIs_final, centroid_xs, centroid_ys)),
              columns=['Scene','BoxID', 'Centroid_X','Centroid_Y'])

#save as
centroids_df.to_csv(path_or_buf = '/home/jukes/Documents/Sample_glaciers/trim_centroids_'+massorsize+'.csv', sep=',')
centroids_df

,Scene,BoxID,Centroid_X,Centroid_Y
0,LC80130022017108LGN00,001,144.740741,159.574074
1,LC80100022015258LGN00,001,169.259259,150.333333
2,LC80120022014269LGN00,001,183.314286,153.714286
3,LC80100022013124LGN01,001,144.859649,158.807018
4,LC80110022014182LGN00,001,144.186441,157.372881
5,LC80080022015116LGN00,001,132.750000,164.090909
6,LC80060032014083LGN00,001,153.207317,160.573171
7,LC80140012014171LGN00,001,132.425000,163.987500
8,LC80120022017101LGN00,001,159.690141,161.098592
9,LC80070022015109LGN00,001,132.586207,163.459770
